In [26]:
import re
import string
import numpy as np
import pandas as pd
from nltk.stem.snowball import SnowballStemmer 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from imblearn.over_sampling import SMOTE
from imblearn import under_sampling
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from sklearn.metrics import accuracy_score, plot_confusion_matrix, classification_report, confusion_matrix

In [38]:
df = pd.read_csv('fake_jobs.csv',index_col=0)

In [39]:
df.head()

,loc,salary_range,has_questions,employment_type,fraudulent,text
0,US,1,0,Other,0,marketing intern were food and weve created a ...
1,NZ,1,0,Full-time,0,customer service cloud video production second...
2,US,1,0,NaN,0,commissioning machinery assistant cma valor se...
3,US,1,0,Full-time,0,account executive washington dc our passion fo...
4,US,1,1,Full-time,0,bill review manager spotsource solutions llc i...


In [40]:
df.fillna('',inplace=True)

In [41]:
enc = LabelEncoder()
df.loc[:,['employment_type','loc']] = df.loc[:,['employment_type','loc']].apply(enc.fit_transform)

In [42]:
tf = TfidfVectorizer(max_features=2000)

In [43]:
df1 = pd.DataFrame(tf.fit_transform(df['text']).toarray(),columns=tf.get_feature_names())
df.drop(['text'],axis=1,inplace=True)
main_df = pd.concat([df1,df],axis=1)

In [44]:
features = main_df.drop(['fraudulent'],axis=1)
target = main_df['fraudulent']

In [45]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train, y_test = train_test_split(features,target,random_state=0,test_size=0.3)
print("Before smote - Train : " ,Counter(y_train))

Before smote - Train :  Counter({0: 11877, 1: 639})


In [46]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

In [47]:
hidden_units=100
learning_rate=0.01
hidden_layer_act='tanh'
output_layer_act='sigmoid'
no_epochs=100

In [48]:
model = Sequential()
model.add(Dense(units=150, activation='tanh'))
model.add(Dense(units=100, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(
    x=x_train,
    y=y_train,
    epochs=100,
    shuffle=True,
    verbose=2,
    batch_size = 20,
)

Epoch 1/100
626/626 - 2s - loss: 0.1400 - accuracy: 0.9569
Epoch 2/100
626/626 - 2s - loss: 0.0694 - accuracy: 0.9762
Epoch 3/100
626/626 - 1s - loss: 0.0477 - accuracy: 0.9829
Epoch 4/100
626/626 - 1s - loss: 0.0364 - accuracy: 0.9868
Epoch 5/100
626/626 - 1s - loss: 0.0295 - accuracy: 0.9896
Epoch 6/100
626/626 - 1s - loss: 0.0187 - accuracy: 0.9940
Epoch 7/100
626/626 - 1s - loss: 0.0180 - accuracy: 0.9942
Epoch 8/100
626/626 - 1s - loss: 0.0162 - accuracy: 0.9943
Epoch 9/100
626/626 - 1s - loss: 0.0178 - accuracy: 0.9942
Epoch 10/100
626/626 - 1s - loss: 0.0138 - accuracy: 0.9950
Epoch 11/100
626/626 - 1s - loss: 0.0133 - accuracy: 0.9955
Epoch 12/100
626/626 - 1s - loss: 0.0082 - accuracy: 0.9972
Epoch 13/100
626/626 - 1s - loss: 0.0094 - accuracy: 0.9963
Epoch 14/100
626/626 - 2s - loss: 0.0093 - accuracy: 0.9970
Epoch 15/100
626/626 - 2s - loss: 0.0105 - accuracy: 0.9974
Epoch 16/100
626/626 - 1s - loss: 0.0128 - accuracy: 0.9957
Epoch 17/100
626/626 - 1s - loss: 0.0152 - accura

In [49]:
pred = model.predict(x_test)

In [50]:
pred = [int(round(x[0])) for x in pred]
confusion_matrix(y_test,pred)

array([[5075,   62],
       [  46,  181]], dtype=int64)

In [51]:
sm = under_sampling.RandomUnderSampler(sampling_strategy='auto',random_state=42)
x_train_under_sample , y_train_under_sample = sm.fit_resample(x_train,y_train)

In [52]:
x_train_under_sample = np.asarray(x_train_under_sample)
y_train_under_sample = np.asarray(y_train_under_sample)

In [53]:
model.fit(
    x=x_train_under_sample,
    y=y_train_under_sample,
    epochs=100,
    shuffle=True,
    verbose=2,
    batch_size = 20,
)
pred = model.predict(x_test)

Epoch 1/100
64/64 - 0s - loss: 6.6541e-09 - accuracy: 1.0000
Epoch 2/100
64/64 - 0s - loss: 5.4290e-09 - accuracy: 1.0000
Epoch 3/100
64/64 - 0s - loss: 4.8975e-09 - accuracy: 1.0000
Epoch 4/100
64/64 - 0s - loss: 4.8243e-09 - accuracy: 1.0000
Epoch 5/100
64/64 - 0s - loss: 4.2236e-09 - accuracy: 1.0000
Epoch 6/100
64/64 - 0s - loss: 4.2089e-09 - accuracy: 1.0000
Epoch 7/100
64/64 - 0s - loss: 3.9551e-09 - accuracy: 1.0000
Epoch 8/100
64/64 - 0s - loss: 3.8042e-09 - accuracy: 1.0000
Epoch 9/100
64/64 - 0s - loss: 3.6786e-09 - accuracy: 1.0000
Epoch 10/100
64/64 - 0s - loss: 3.4667e-09 - accuracy: 1.0000
Epoch 11/100
64/64 - 0s - loss: 3.5152e-09 - accuracy: 1.0000
Epoch 12/100
64/64 - 0s - loss: 3.3047e-09 - accuracy: 1.0000
Epoch 13/100
64/64 - 0s - loss: 3.2272e-09 - accuracy: 1.0000
Epoch 14/100
64/64 - 0s - loss: 3.1214e-09 - accuracy: 1.0000
Epoch 15/100
64/64 - 0s - loss: 3.0267e-09 - accuracy: 1.0000
Epoch 16/100
64/64 - 0s - loss: 2.8730e-09 - accuracy: 1.0000
Epoch 17/100
64/6

In [54]:
pred = model.predict(x_test)
pred = [int(round(x[0])) for x in pred]
confusion_matrix(y_test,pred)

array([[5042,   95],
       [  41,  186]], dtype=int64)